In [19]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import warnings
warnings.filterwarnings('ignore')

openai.api_key  = os.getenv('OPENAI_API_KEY')
#print(os.getenv('OPENAI_API_KEY'))

In [20]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [21]:
# Inizializing the context with the first directive
#context.clear()
context = [ {'role':'system', 'content':"""You are an assistant helping the user spell city names correctly. \
You first greet the customer, then ask for the departure and arrival city, \
If the user types the name of a city incorrectly, try to guess what's the correct one \
You need to get the following information from the user: \
- Departure City \
- Arrival City \
If the user types the name of a city incorrectly, try to guess what's the correct one. \
You respond in a short, very conversational friendly style. \
Once you collect all the information output the name of the departure and arrival cities in the following string format \
No additional text or comment \
https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=[Departure City]&limit=10 \
https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=[Arrival City]&limit=10 \
You only answer to question related to your duty.
"""} ] 
print(context)

[{'role': 'system', 'content': "You are an assistant helping the user spell city names correctly. You first greet the customer, then ask for the departure and arrival city, If the user types the name of a city incorrectly, try to guess what's the correct one You need to get the following information from the user: - Departure City - Arrival City If the user types the name of a city incorrectly, try to guess what's the correct one. You respond in a short, very conversational friendly style. Once you collect all the information output the name of the departure and arrival cities in the following string format No additional text or comment https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=[Departure City]&limit=10 https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=[Arrival City]&limit=10 You only answer to question related to your duty.\n"}]


In [22]:
#!pip install requests

import re
import requests

def analyze_for_stations(prompt):

    # The regex pattern for the specific URL
    pattern = r"https://www\.lefrecce\.it/Channels\.Website\.BFF\.WEB/website/locations/search\?name=[^\s]+"
    urls = re.findall(pattern, prompt)
    
    # if there are urls in the prompt, it means the model provided the user
    # the correct names of the departure and arrival stations
    # we need to invole trenitalia API to retrieve stations lists
    if len(urls) > 1:
        # assign the URLs to two different variables
        departure_url = urls[0] if len(urls) > 0 else None
        arrivals_url = str(urls[1]) if len(urls) > 1 else None

        # retrieves the departure stations
        resp = requests.get(departure_url)
        if resp.status_code == 200:
            departure_stations = resp.text
#            print(departure_stations)

        # retrieves the arrival stations
        resp = requests.get(arrivals_url)
        if resp.status_code == 200:
            arrival_stations = resp.text
#            print(arrival_stations)

        # once we have the station list it's time to add the second
        # context to the API
        context.append({'role':'system', 'content':"""You are an assistant from Trenitalia helping the user identify the correct departure train station \
        and the correct arrival train station \ 
        The departure train stations list is in the following JSON"""+ 
        str({departure_stations}) +
        """The arrival train stations list is in the following JSON"""+ 
        str({arrival_stations}) +
        """List the train stations by displayName. \
        Ask to identify the departure and arrival stations. \
        Once you collect all the information output the corresponding id in a list. \
        No additional text or comment. \
        """})
    
        # we aknowledge that we found the cities
        return True
    else:
        return False

In [23]:
#analyze_for_stations("Great! Here are the links to search for train stations in Milan and Bologna: Departure City: https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=Milan&limit=15 Arrival City: https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=Bologna&limit=15 I hope this helps with your travel plans!")
#print(context)
#analyze_for_stations("Hello! How can I assist you today with your travel plans? User: Hi! I need help with spelling the names of the cities I want to travel to. Assistant: Sure thing! What is your departure city? User: My departure city is Sienaa. Assistant: Did you mean Siena, Italy? User: Yes, that's what I meant. Assistant: Great! And what is your arrival city? User: My arrival city is Florance. Assistant: Did you mean Florence, Italy? User: Yes, that's the one. Assistant: Perfect! Here are the links to search for train schedules between Siena and Florence on the Trenitalia websit")


In [24]:
#!pip install requests
import re
import requests
import requests
import json
from datetime import datetime, timezone, timedelta

def analyze_for_ids(prompt):


    # find all sequences of 9 digits
    ids = re.findall(r'\b\d{9}\b', prompt)
    
    # if there are ids in the prompt, it means the model provided the user
    # the correct departur and arrival station ids
    # we need to invole trenitalia API to retrieve tran solutions
    if len(ids) > 1:
        # assign the URLs to two different variables
        departure_id = ids[0] if len(ids) > 0 else None
        arrival_id = str(ids[1]) if len(ids) > 1 else None

        print(departure_id)
        print(arrival_id)
        print(datetime.now(timezone(timedelta(hours=2))).replace(microsecond=0).isoformat())
        
        # Get the current date and time, with timezone
#       now = datetime.now(timezone(timedelta(hours=2)))
        # Format it with microseconds set to zero
#       formatted_now = now.replace(microsecond=0).isoformat()

#       print(formatted_now)
        
        json_data = { 
            "departureLocationId": departure_id, 
            "arrivalLocationId": arrival_id, 
            "departureTime": datetime.now(timezone(timedelta(hours=2))).replace(microsecond=0).isoformat(), 
            "adults": 1,
            "children": 0,
            "criteria": {
                "frecceOnly": False,
                "regionalOnly": False,
                "noChanges": False,
                "order": "DEPARTURE_DATE",
                "limit": 5,
                "offset": 0
            },
            "advancedSearchRequest": {
            "bestFare": False
            }
        }
        
        print(json_data)
        
        
        url = 'https://www.lefrecce.it/Channels.Website.BFF.WEB/website/ticket/solutions'
        headers = {"Content-Type": "application/json; charset=utf-8"}
        response = requests.post(url, headers=headers, json = json_data)
        #print(response.text)
        # assuming response is your JSON data from the POST request
        data = json.loads(response.text)

        solutions = data['solutions']

        extracted_data = []
        for solution in solutions:
            new_dict = {}
            new_dict['id'] = solution['solution']['id']
            new_dict['origin'] = solution['solution']['origin']
            new_dict['destination'] = solution['solution']['destination']
            new_dict['departureTime'] = datetime.fromisoformat(solution['solution']['departureTime'].replace("Z", "+00:00")).strftime("%d.%m.%Y %H:%M")
            new_dict['arrivalTime'] = datetime.fromisoformat(solution['solution']['arrivalTime'].replace("Z", "+00:00")).strftime("%d.%m.%Y %H:%M")
            new_dict['duration'] = solution['solution']['duration']

            if solution['solution']['price']:
                new_dict['amount'] = solution['solution']['price']['amount']

            # assuming that there is only one train in the nodes list, 
            # otherwise you will need to loop through the nodes list
            if solution['solution']['nodes']:
                new_dict['trainCategory'] = solution['solution']['nodes'][0]['train']['trainCategory']
        
            extracted_data.append(new_dict)
            
            # once we have the station list it's time to add the second
            # context to the API
            context.append({'role':'system', 'content':"""
                You are an assistant From Trenitalia helping the user identify the correct  train solution that best fits user's needs \
                The trains solutions are listed in the following JSON"""+ 
                str({json.dumps(extracted_data)}) +
                """You respond in a short, very conversational friendly style. \
                List 'origin','destination','departureTime','arrivalTime', 'duration', 'trainCategory' 'amount' in a tabular format \
                You wait to understand which is the solution the customer wants \
                Finally you collect the payment.\
                """}) # accumulate messages
        return True
    else:
        return False


In [25]:
print(analyze_for_ids("The departure station is Milano Centrale and the arrival station is Firenze S. M. Novella. Here are the corresponding ids for each station: Departure Station (Milano Centrale): 830001700 Arrival Station (Firenze S. M. Novella): 830006421 You can use these ids to search for train schedules and purchase tickets on the Trenitalia website."))



830001700
830006421
2023-06-15T16:37:24+02:00
{'departureLocationId': '830001700', 'arrivalLocationId': '830006421', 'departureTime': '2023-06-15T16:37:24+02:00', 'adults': 1, 'children': 0, 'criteria': {'frecceOnly': False, 'regionalOnly': False, 'noChanges': False, 'order': 'DEPARTURE_DATE', 'limit': 5, 'offset': 0}, 'advancedSearchRequest': {'bestFare': False}}
True


In [26]:
def collect_messages(_):
    
    # gets user's prompt from the bot
    prompt = inp.value_input
    
    # clears the input from the bot
    inp.value = ''
    
  
    # appends user's prompt to the context
    # cntext contains the full history of the conversation
    # every time needs to be passed back to the API 
    context.append({'role':'user', 'content':f"{prompt}"})
    
    # sends the new contex (with the new prompt)
    # to the API
    response = get_completion_from_messages(context) 

    # analazyes the context to see if we need
    # to add the select list of stations ridective
    # if analyze_prompt(response) == False:
    if analyze_for_stations(response):
        response = get_completion_from_messages(context) 
        #panels.append(pn.Row('User:', pn.pane.Markdown(str(context), width=600)))
        # if the model doesnt find the list of the cities 
        # We apend API answer to the context so to the next iteration can be send back to the API
        # otherwise we do not append the respose because is a message we use to esxtract the url
        # to make a call to the trenitalia APIs
    elif analyze_for_ids(response):
        response = get_completion_from_messages(context) 
        #panels.append(pn.Row('User:', pn.pane.Markdown(str(context), width=600)))
    else:
        context.append({'role':'assistant', 'content':f"{response}"})

    # usefull to debug
    # panels.append(pn.Row('User:', pn.pane.Markdown(str(context), width=600)))
        
    # Adds user's prompt and API answer to the UI so the converastion is displayed in the history
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    # returns the panes to the bot UI component
    return pn.Column(*panels)

In [29]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context.clear()
context = [ {'role':'system', 'content':"""You are an assistant helping the user spell city names correctly. \
You first greet the customer, then ask for the departure and arrival city, \
If the user types the name of a city incorrectly, try to guess what's the correct one \
You need to get the following information from the user: \
- Departure City \
- Arrival City \
If the user types the name of a city incorrectly, try to guess what's the correct one. \
You respond in a short, very conversational friendly style. \
Once you collect all the information output the name of the departure and arrival cities in the following string format \
No additional text or comment \
https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=[Departure City]&limit=10 \
https://www.lefrecce.it/Channels.Website.BFF.WEB/website/locations/search?name=[Arrival City]&limit=10 \
You only answer to question related to your duty.
"""} ] 

# creates a bot gui componenet where the user can enter the prompt
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')

# create a bot gui compoenet button to chat 
button_conversation = pn.widgets.Button(name="Chat!")

# binds the button "chat" with the method collect_messages
# this way when the user clicks the butto the "collect_messages" method is invoked
interactive_conversation = pn.bind(collect_messages, button_conversation)

# creates the chat it adds:
# - the input text (inp)
# - 
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=600),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=600, loading_indicator=True)